<center>
<img src="https://upload.wikimedia.org/wikipedia/fr/thumb/1/1d/Logo_T%C3%A9l%C3%A9com_SudParis.svg/1014px-Logo_T%C3%A9l%C3%A9com_SudParis.svg.png" width="10%" />
</center>

<center> <h2> NET 4103/7431 Complex Network </h2> </center>

<center> <h3> Vincent Gauthier (vincent.gauthier@telecom-sudparis.eu) </h3> </center>

### Note
Avant de commencer les exercices, assurez-vous que tout fonctionne comme prévu. Tout d'abord, le redémarrage du kernel **(dans la barre de menus, sélectionnez le kernel $\rightarrow$ Restart)**.

Assurez-vous que vous remplir les célluler aux endroits marquer «YOUR CODE HERE». 

Veuillez supprimer les ligne «raise NotImplementedError()» dans toutes les cellules auxquelles vous avez répondu, ainsi que votre nom et prénom ci-dessous:

In [ ]:
NOM = "XXX"
PRENOM = "XXX"

---

<h1 align="center">TP 3: Algorithmes de detection de communautés</h1> 
<h1 align="center">LAB 3: Community detection Algorithms</h1> 
</br>
</br>
</br>
<img src="../../images/community.svg" style="display:block;margin-left:auto;margin-right:auto;width:50%;"></img>

In [ ]:
!pip install sklearn
!pip install neo4j-driver
!pip install scikit-network

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np

import sknetwork as skn
from sklearn.metrics.cluster import normalized_mutual_info_score

%matplotlib inline

# Style pour le Notebook
from IPython.core.display import HTML

def css_styling():
    styles = open("../../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [ ]:
import networkx as nx
from packaging import version
import sys 
import sknetwork as skn

print("Python version:", sys.version)
print("networkx version:", nx.__version__)

# assert networkx version is greater or equal to 2.6
assert version.parse(nx.__version__) >= version.parse("2.6")

# assert python version is greater that 3.7
assert sys.version_info[0] == 3
assert sys.version_info[1] >= 7  

# assert sknetwork version is greater or equal to 0.24
assert version.parse(skn.__version__) >= version.parse("0.24")

# If working in colab mount the drive filesystem 
if 'google.colab' in str(get_ipython()):
    print('Working in colab')
    
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("working locally")

## (EN)
## Zachary's karate club ([Wikipedia](https://en.wikipedia.org/wiki/Zachary%27s_karate_club#cite_note-Zachary-2))

Zachary's karate club is a social network of a university karate club, described in the paper "An Information Flow Model for Conflict and Fission in Small Groups" by Wayne W. Zachary. The network became a popular example of community structure in networks after its use by Michelle Girvan and Mark Newman in 2002.

A social network of a karate club was studied by Wayne W. Zachary for a period of three years from 1970 to 1972 __[Z77]__. The network captures 34 members of a karate club, documenting links between pairs of members who interacted outside the club. During the study a conflict arose between the administrator "John A" and instructor "Mr. Hi" (pseudonyms), which led to the split of the club into two. Half of the members formed a new club around Mr. Hi; members of the other part found a new instructor or gave up karate. Based on collected data Zachary correctly assigned all but one member of the club to the groups they actually joined after the split.

### References

__[Z77]__
Zachary, W. W. (1977). An Information Flow Model for Conflict and Fission in Small Groups. Journal of Anthropological Research. 33 (4): 452–473.

## (FR)
## Le club de karaté de Zachary ([Wikipedia](https://en.wikipedia.org/wiki/Zachary%27s_karate_club#cite_note-Zachary-2))

Le club de karaté de Zachary est un réseau social d'un club de karaté universitaire, décrit dans l'article «Un modèle de flux d'information pour les conflits et la fission en petits groupes» de Wayne W. Zachary. Le réseau est devenu un exemple populaire de structure communautaire dans les réseaux après son utilisation par Michelle Girvan et Mark Newman en 2002. 

Un réseau social d'un club de karaté universitaire a été étudié par Wayne W. Zachary pendant une période de trois ans de 1970 à 1972 __[Z77]__. Le réseau capture 34 membres d'un club de karaté, documentant les liens entre des paires de membres qui ont interagi en dehors du club. Au cours de l'étude, un conflit est survenu entre l'administrateur «John A» et l'instructeur «M. Hi» (pseudonymes), ce qui a conduit à la scission du club en deux. La moitié des membres ont formé un nouveau club autour de M. Hi; les membres de l'autre partie ont trouvé un nouvel instructeur ou ont abandonné le karaté. Sur la base des données collectées, Zachary a correctement attribué tous les membres du club sauf un aux groupes qu'ils ont effectivement rejoints après la scission.

### Références

__[Z77]__
Zachary, W. W. (1977). An Information Flow Model for Conflict and Fission in Small Groups. Journal of Anthropological Research. 33 (4): 452–473.

In [ ]:
from networkx.drawing.layout import spring_layout

def plot_graph(G, node_color, colormap="RdBu_r", ax=None):
    dpi = 300
    h,w = 2480,3508

    pos = spring_layout(G, weight="weight")
    #pos = spring_layout(G)


    fig = plt.figure(figsize=(w/dpi,h/dpi),dpi=dpi)
    ax = plt.gca()

    nx.draw_networkx_edges(G, pos, alpha=0.3)
    nx.draw_networkx_nodes(
        G,
        pos,
        node_size=300,
        edgecolors="white",
        node_color=node_color,
        cmap=colormap,
    )

    nx.draw_networkx_labels(G, pos, font_size=7, font_color="white")

    ax.margins(0.20)
    plt.axis("off")
    plt.show()

In [ ]:
G = nx.karate_club_graph()

node_degree = list(dict(G.degree()).values())
node_color = (np.array(node_degree) - np.mean(node_degree))/np.std(node_degree)

plot_graph(G, node_color=node_color)

## Exercice 1: Label propagation algorithm (LPA)

### (EN)

The Label Propagation algorithm (LPA) is a fast algorithm for finding communities in a graph. It detects these communities using network structure alone as its guide, and doesn’t require a predefined objective function or prior information about the communities.

LPA is a relatively new algorithm, and was only proposed by *Raghavan et al* in 2007, in __[RAK07]__. It works by propagating labels throughout the network and forming communities based on this process of label propagation.

The intuition behind the algorithm is that a single label can quickly become dominant in a densely connected group of nodes, but will have trouble crossing a sparsely connected region. Labels will get trapped inside a densely connected group of nodes, and those nodes that end up with the same label when the algorithms finish can be considered part of the same community.

The algorithm works as follows:

1. Every node is initialized with a unique community label (an identifier).
2. These labels propagate through the network.
3. At every iteration of propagation, each node updates its label to the one that the maximum numbers of its neighbors belong to. Ties are broken uniformly and randomly.
4. LPA reaches convergence when each node has the majority label of its neighbors.
5. LPA stops if either convergence or the user-defined maximum number of iterations is achieved.

As labels propagate, densely connected groups of nodes quickly reach a consensus on a unique label. At the end of the propagation only a few labels will remain - most will have disappeared. Nodes that have the same community label at convergence are said to belong to the same community.

### 1.1) Implement the Label propagation algorithm (LPA)

* __Step 1__: give a unique label to each node in the network
* __Step 2__: Arrange the nodes in the network in a random order, please use the Fisher-Yates algorithm __[FY]__ to shuffle the nodeID in random order
* __Step 3__: for each node in the network (in this random order) set its label to a label occurring with the highest frequency  among its neighbors (break ties uniformly)
* __Step 4__ : go to 2 as long as there exists a node with a label that does not have the highest frequency among its neighbors.

### References

__[RAK07]__
Raghavan, U. N., Albert, R., & Kumara, S. (2007). Near linear time algorithm to detect community structures in large-scale networks. Physical Review E - Statistical, Nonlinear, and Soft Matter Physics. [10.1103/PhysRevE.76.036106](https://doi.org/10.1103/PhysRevE.76.036106)

__[FY]__ 
[Fisher-Yates shuffle](https://en.wikipedia.org/wiki/Fisher-Yates_shuffle)

### (FR)

L'algorithme de propagation d'étiquette (Label Propagation Algorithm or LPA) est un algorithme rapide pour trouver des communautés dans un graph. Il détecte ces communautés en utilisant uniquement la structure du réseau comme guide et ne nécessite pas de fonction d'objectif prédéfinie ou d'informations préalables sur les communautés.

LPA est un algorithme relativement nouveau, et n'a été proposé que par *Raghavan et al* en 2007, dans __[RAK07]__. Il fonctionne en propageant les étiquettes à travers le réseau et en formant des communautés basées sur ce processus de propagation des étiquettes.

L'intuition derrière l'algorithme est qu'une seule étiquette peut rapidement devenir dominante dans un groupe de nœuds densément connectés, mais aura du mal à traverser une région faiblement connectée. Les étiquettes seront piégées dans un groupe de nœuds densément connectés, et les nœuds qui se retrouveront avec la même étiquette une fois les algorithmes terminés peuvent être considérés comme faisant partie de la même communauté.

L'algorithme fonctionne comme suit:
1. Chaque noeud est initialisé avec une étiquette de communauté unique (un identifiant).
2. Ces étiquettes se propagent à travers le réseau, on sélectionne.
3. À chaque itération de propagation, chaque noeud met à jour son étiquette avec celle à laquelle appartient le nombre maximum de ses voisins. Les égalités sont rompues de manière uniforme et aléatoire.
4. LPA atteint la convergence lorsque chaque noeud a l'étiquette majoritaire de ses voisins.
5. LPA s'arrête si la convergence ou le nombre maximal d'itérations défini par l'utilisateur est atteint.

À mesure que les étiquettes se propagent, des groupes de noeuds densément connectés parviennent rapidement à un consensus sur une étiquette unique. À la fin de la propagation, il ne restera que quelques étiquettes - la plupart auront disparu. On dit que les nœuds qui ont le même label de communauté à la convergence appartiennent à la même communauté.

### 1.1) Implémentation de l'algorithme LPA

* __Étape 1__: attribuez une étiquette unique à chaque noeud du réseau
* __Étape 2__: Disposez les noeuds du réseau dans un ordre aléatoire, veuillez utiliser l'algorithme de Fisher-Yates __[FY]__ pour mélanger  l’ID des noeuds. aléatoirement
* __Étape 3__: pour chaque noeud du réseau (dans cet ordre aléatoire), définissez son étiquette sur une étiquette se produisant avec la fréquence la plus élevée parmi ses voisins
* __Étape 4__: passez à 2 tant qu'il existe un nœud avec une étiquette qui n'a pas la fréquence la plus élevée parmi ses voisins. 

### 1.2) afficher le résultat l'algorithme LPA avec le réseau du karaté club

### Références

__[RAK07]__
Raghavan, U. N., Albert, R., & Kumara, S. (2007). Near linear time algorithm to detect community structures in large-scale networks. Physical Review E - Statistical, Nonlinear, and Soft Matter Physics. [10.1103/PhysRevE.76.036106](https://doi.org/10.1103/PhysRevE.76.036106)

__[FY]__ 
[Fisher-Yates shuffle](https://en.wikipedia.org/wiki/Fisher-Yates_shuffle)

In [ ]:
def fisher_yates(arr):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
def lpa(G, max_iter=100):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
community_dict = lpa(G)

In [ ]:
dpi = 300
h,w = 2480,3508
fig = plt.figure(figsize=(w/dpi,h/dpi),dpi=dpi)
ax = plt.gca()


pos = spring_layout(G)

nx.draw_networkx_edges(G, pos, alpha=0.3)
nx.draw_networkx_nodes(
        G,
        pos,
        node_color=[community_dict[node_id] for node_id in list(G)],
        edgecolors="white",
        alpha=0.7,
        cmap="RdBu_r"
)
nx.draw_networkx_labels(
                        G, 
                        pos, 
                        labels={ n_id:G.nodes[n_id]["club"] for n_id in G.nodes()}, 
                        font_size=9, 
                        font_color="black");
plt.axis("off")
plt.show()

### (EN)
### 1.3) Discuss the results obtained by the LPA algorithm on the "Zachary's karate club"

Your Answer ?

### (FR)
### 1.3) Discuter les resultats obtenue par l'algorithme LPA sur le graph du "Karaté club de Zachary"

Your Answer ?

## (FR)
## Exercice 2: Validation d'un algorithme de detection de communautés
En s'inspirant de __[GIR02]__ nous allons construirte un benchmark pour les algorithmes de détection de communautés. Le modèle de bloc stochastique est un modèle génératif pour les graphes aléatoires. Ce modèle tend à produire des graphes contenant des communautés (des sous-ensembles caractérisés par le fait d'être connectés les uns aux autres avec des densités d'arêtes particulières). Par exemple, les lisières peuvent être plus fréquentes au sein des communautés qu'entre les communautés. Le modèle de bloc stochastique est important dans les statistiques, l'apprentissage automatique et la science des réseaux, où il sert de référence.

Implémentez un algorithme pour générer le graphique aléatoire suivant.

- Le graphe comporte 400 noeuds répartis en 4 clusters de taille 100.
- Chaque paire de noeuds dans le même cluster est connectée avec une probabilité $p$.
- Chaque paire de noeuds dans différents clusters est connectée avec une probabilité $q < p$. Afficher les graphes obtenus pour différentes valeurs de $p$ et $q$ à l'aide d'un logiciel de votre choix ([Networkx](https://networkx.github.io/documentation/stable/reference/drawing)). Quelle est l'effet du rapport $\frac{p}{q}$ sur la structure des communautés ?

__[GIR02]__ M. Girvan, M. E. J. Newman, Community structure in social and biological networks, In Proc. of the National Academy of Sciences Jun 2002, 99 (12) 7821-7826. [(HTML)](https://www.pnas.org/content/99/12/7821)

## (EN)
## Exercice 2: Validation of the community detection algorithm

Inspired by the work of __[GIR02]__ we will build a benchmark for community detection algorithms. The stochastic block model is a generative model for random graphs. This model tends to produce graphs containing communities, subsets characterized by being connected with one another with particular edge densities. For example, edges may be more common within communities than between communities. The stochastic block model is important in statistics, machine learning, and network science, where it serves as a useful benchmark for the task of recovering community structure in graph data.

Implement an algorithm to generate the following random graph.
- The graph has 400 nodes partitioned into 4 clusters of size 100.
- Each pair of nodes in the same cluster is connected with a probability $p$.
- Each pair of nodes in different clusters is connected with a probability $q < p$.
Draw the obtained graphs for various values of $p$ and $q$ using a software of your choice. For instance: [Networkx](https://networkx.github.io/documentation/stable/reference/drawing). 
What is the effect of increasing or decreasing $\frac{p}{q}$ on the community structure?


__[GIR02]__ M. Girvan, M. E. J. Newman, Community structure in social and biological networks, In Proc. of the National Academy of Sciences Jun 2002, 99 (12) 7821-7826. [(HTML)](https://www.pnas.org/content/99/12/7821)

In [ ]:
def stochastic_block(n, n_community, p, q):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
g1, Ycommunity_true = stochastic_block(400, n_community=2, p=0.1, q=0.001)
plot_graph(g1, node_color=Ycommunity_true)
plt.tight_layout()
plt.show()

In [ ]:
g2, Ycommunity_true = stochastic_block(400, n_community=4, p=0.1, q=0.001)
plot_graph(g2, node_color=Ycommunity_true)

plt.tight_layout()
plt.show()

In [ ]:
n_simulation = 30
p = 0.1
result = []
for n_community in [2, 4, 6, 8]:
    for q in np.linspace(0.001, 0.03, 20):
        mmi = []
        for _ in range(n_simulation):
            g, Ycommunity = stochastic_block(400, n_community=n_community, p=p, q=q)        
            Ycommunity_dict = lpa(g)
            Ycommunity_hat = list(Ycommunity_dict.values())
            mmi.append(normalized_mutual_info_score(Ycommunity, Ycommunity_hat))
        result.append((n_community, q, np.mean(mmi)))

## Plots the results 

In [ ]:
df = pd.DataFrame(result, columns=["n_community", "q", "mmi"])

In [ ]:
fig, ax = plt.subplots()
ax.semilogx(df[df.n_community == 2].q.values, df[df.n_community == 2].mmi.values, "o-", label="n_community=2")
ax.semilogx(df[df.n_community == 4].q.values, df[df.n_community == 4].mmi.values, "o-", label="n_community=4")
ax.semilogx(df[df.n_community == 6].q.values, df[df.n_community == 6].mmi.values, "o-", label="n_community=6")
ax.semilogx(df[df.n_community == 8].q.values, df[df.n_community == 8].mmi.values, "o-", label="n_community=8")

ax.set_xlabel(r"q")
ax.set_ylabel("Normalizd mutual information")
ax.legend()
plt.tight_layout()
plt.show()

# (EN)
# Exercice 3: Implementation of the Newman/Girvan community detection algorithm


The algorithm's described in __[NEW04]__ steps for community detection are summarized below

- The betweenness of all existing edges in the network is calculated first.
- The edge(s) with the highest betweenness are removed.
- The betweenness of all edges affected by the removal is recalculated.
- Steps 2 and 3 are repeated until no edges remain.

__[NEW04]__  Newman and Girvan, Finding and evaluating community structure in networks, Physical review E, 2004. [(arXiv)](https://arxiv.org/abs/cond-mat/0308217)


# (FR)
# Exercice 3 : Implémentation de l'algorithme de détection de communautés de Newman/Girvan

Les algorithmes décrits dans les étapes suivantes __[NEW04]__ résumés ci-dessous

- On calcule la betweenness de toutes les arêtes existantes dans le réseau.
- le lien avec la betweenness la plus élevée est supprimé du graph.
- On recalcule la betweenness de toutes les arêtes restante dans le réseau.
- Les étapes 2 et 3 sont répétées jusqu'à ce que le nombre de communautés désirer soit atteint.

__[NEW04]__  Newman and Girvan, Finding and evaluating community structure in networks, Physical review E, 2004. [(arXiv)](https://arxiv.org/abs/cond-mat/0308217)

In [ ]:
def newman_girvan(G, n_community=2):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
G = nx.karate_club_graph()
nodes = newman_girvan(G)

# save the community assignment in the graph structure
G = nx.karate_club_graph()
for communityId, community in enumerate(nodes):
    for nodeId in community:
        G.nodes[nodeId]["community"] = communityId

In [ ]:
nodes

## 3.2 Plot the community structure of the Karate club social graph according the Newman/Girvan algorithm

In [ ]:
dpi = 300
h,w = 2480,3508
fig = plt.figure(figsize=(w/dpi,h/dpi),dpi=dpi)
ax = plt.gca()


pos = spring_layout(G)

nx.draw_networkx_edges(G, pos, alpha=0.3)
nx.draw_networkx_nodes(
        G,
        pos,
        node_color=[G.nodes[node_id]['community'] for node_id in list(G)],
        edgecolors="white",
        alpha=0.7,
        cmap="RdBu_r"
)
nx.draw_networkx_labels(
                        G, 
                        pos, 
                        labels={ n_id:G.nodes[n_id]["club"] for n_id in G.nodes()}, 
                        font_size=9, 
                        font_color="black");
plt.axis("off")
plt.show()

## (EN)
## 3.3: Discuss the results obtained by the Newman/Girvan algorithm on the "Zachary's karate club"

Your Answer ?

## (FR)
## 3.3 Discuter les résultats obtenus par l'algorithme Newman/Girvan sur le graph du "Karaté club de Zachary"

Votre réponse ?

# (EN) 
# Exercice 4: Finding Community in retweet network

## The Russian Twitter Trolls Database In Neo4j

As part of the House Intelligence Committee investigation into how Russia may have influenced the 2016 US Election, Twitter released the screen names of almost 3000 Twitter accounts believed to be connected to Russia’s Internet Research Agency, a company known for operating social media troll accounts. Twitter immediately suspended these accounts, deleting their data from Twitter.com and the Twitter API.

A team at NBC News including **Ben Popken** and **EJ Fox** was able to reconstruct a dataset consisting of a subset of the deleted data for their investigation and using Neo4j were able to show how these troll accounts went on attack during key election moments. NBC News open-sourced the reconstructed dataset and released it as this Neo4j database.

NBC News has publicly released a database of deleted tweets from their investigation into how Russian Twitter Trolls may have influenced the 2016 US election. You can read about the results of NBC’s analysis in their stories [here](https://www.nbcnews.com/tech/social-media/now-available-more-200-000-deleted-russian-troll-tweets-n844731) and [here](https://www.nbcnews.com/tech/social-media/russian-trolls-went-attack-during-key-election-moments-n827176), but the focus of this post will be on how you can explore the data on your own, using open source data analysis tools. We’ll show how to get started with the data and hopefully inspire you to dig into the data yourself.

The dataset contains: 345K tweets, 41K users (in which 445 users have been identified as Russian trolls)

### Example of a troll account

This screenshot is the profile image of user **@LeroyLovesUSA**, one of the accounts Twitter has identified as being operated by the Internet Research Agency in Russia. Many accounts were intended to appear as normal everyday Americans just like this one.

<img src="../../images/russian-troll-twitter-fake-avatar.png" style="display:block;margin-left:auto;margin-right:auto;width:25%;"></img>

In this exercise you will learn to find what:

- Find influential Troll accounts?
- Find communities using community detection algorithms


## Neo4j Sandbox and Neo4j Browser
NBC News has released the data as a Neo4j Database and CSV files that can be used with your favorite data analysis tools. But the easiest way to get started with the data is by using [Neo4j Sandbox](https://neo4j.com/sandbox/). [Neo4j Sandbox](https://neo4j.com/sandbox/) allows you to spin up a private hosted instance of Neo4j pre-populated with interesting datasets.


### Nodes
Nodes can have one or more node labels to describe the "type" of the node.

* Tweet - The tweet
* User - A Twitter user
* Troll - An optional node label to identify users that are on the House list of Russian troll accounts
* Hashtag - Any hashtags used in the tweet
* Source - The Twitter application used to post the tweet.
* URL - A link embedded in a tweet

### Relationships
Relationships have a single relationship type.

* POSTED - Connects the tweet with the User who posted it
* MENTIONS - Connects any Users that are @-mentioned to the Tweet
* HAS_TAG - Connect a tweet to any Hashtag nodes included in the tweet
* POSTED_VIA - Connects a tweet to its Source application node
* RETWEETED - Connects a tweet that retweets another Tweet
* IN_REPLY_TO - Connects a tweet that is a reply to another Tweet

<img src="../../images/troll_datamodel.png" style="display:block;margin-left:auto;margin-right:auto;width:50%;"></img>


# (FR) 
# Exercice 4 : Trouver une communauté dans le réseau de retweet

## La base de données russe des Trolls Twitter dans Neo4j

Dans le cadre de l'enquête du Sénat américain sur la manière dont la Russie a pu influencer les élections américaines de 2016, Twitter a publié les noms d'écran de près de 3000 comptes Twitter qui seraient connectés à l'Internet Research Agency de Russie, une société connue pour exploiter des comptes de trolls sur les réseaux sociaux. Twitter a immédiatement suspendu ces comptes, supprimant leurs données de leur API.

Une équipe de NBC News comprenant **Ben Popken** et **EJ Fox** a pu reconstruire un ensemble de données composé d'un sous-ensemble des données supprimées pour leur enquête et en utilisant Neo4j, ils ont pu montrer comment ces comptes de trolls ont été attaqués pendant les moments clés des élections. NBC News a ouvert l'ensemble des données reconstruites et l'a publié sous le nom de base de données Neo4j.

NBC News a publié publiquement une base de données de tweets supprimés de leur enquête sur la façon dont les Trolls russes de Twitter ont pu influencer les élections américaines de 2016. Vous pouvez lire les résultats de l'analyse de NBC dans leurs histoires [ici](https://www.nbcnews.com/tech/social-media/now-available-more-200-000-deleted-russian-troll-tweets-n844731) et [ici](https://www.nbcnews.com/tech/social-media/russian-trolls-went-attack-during-key- Election-moments-n827176), mais cet article se concentrera sur la façon dont vous pouvez explorer les données par vous-même, à l'aide d'outils d'analyse de données open source. Nous vous montrerons comment démarrer avec les données et, espérons-le, vous inspirerons à creuser vous-même les données.

L'ensemble de données contient : 345K tweets, 41K utilisateurs (dont 445 utilisateurs ont été identifiés comme des trolls russes)

### Exemple de compte troll

Cette capture d'écran est l'image de profil de l'utilisateur **@LeroyLovesUSA**, l'un des comptes identifiés par Twitter comme étant exploité par l'Agence de recherche Internet en Russie. De nombreux comptes étaient destinés à apparaître comme des Américains ordinaires comme celui-ci.

<img src="../../images/russian-troll-twitter-fake-avatar.png" style="display:block;margin-left:auto;margin-right:auto;width:25%;" ></img>

Dans l'exercice, vous apprendrez à trouver ce que :

- Trouver des comptes Troll influents ?
- Trouver des communautés à l'aide d'algorithmes de détection de communauté


## Neo4j Sandbox et navigateur Neo4j
NBC News a publié les données sous forme de base de données Neo4j et de fichiers CSV pouvant être utilisée avec vos outils d'analyse de données préférés. Mais le moyen le plus simple de commencer avec les données est d'utiliser [Neo4j Sandbox] (https://neo4j.com/sandbox/). [Neo4j Sandbox](https://neo4j.com/sandbox/) vous permet de créer une instance hébergée privée de Neo4j préremplie avec des ensembles de données intéressants.

### noeuds
Les noeuds peuvent avoir une ou plusieurs étiquettes pour décrire le "type" du noeud.

* Tweet - Le Tweet
* Utilisateur - Un utilisateur de Twitter
* Troll - Une étiquette de nœud facultative pour identifier les utilisateurs qui figurent sur la liste House des comptes de trolls russes
* Hashtag - Tous les hashtags utilisés dans le tweet
* Source - L'application Twitter utilisée pour publier le tweet
* URL - Un lien intégré dans un tweet

### Des relations
Les relations ont un seul type de relation.

* PUBLIÉ - Connecte le Tweet à l'Utilisateur qui l'a posté.
* MENTIONS - Connecte tous les utilisateurs qui sont @-mentionnés au Tweet.
* HAS_TAG - Connectez un Tweet à n'importe quel nœud Hashtag inclus dans le tweet.
* POSTED_VIA - Connecte un Tweet à son nœud d'application Source.
* RETWEETED - Connecte un Tweet qui retweete un autre Tweet.
* IN_REPLY_TO - Connecte un Tweet qui est une réponse à un autre Tweet.

<img src="../../images/troll_datamodel.png" style="display:block;margin-left:auto;margin-right:auto;width:50%;"></img>

## (EN)
## Connect to the NEO4J database

## (FR)
## Connection à la base de données NEO4J

In [ ]:
from neo4j import GraphDatabase, basic_auth

def query_neo4j(driver, cypher_query, database="neo4j"):
    with driver.session(database=database) as session:
        results = session.read_transaction(
            lambda tx: tx.run(cypher_query).data())
        return results

In [ ]:
#
# Replace the XXX with your own connection credentials
#

driver = GraphDatabase.driver(
    "bolt://3.239.201.198:7687",
    auth=basic_auth("neo4j", "coordination-revolutions-hardcopy"))

## (EN)
## What are the most commonly used hashtags by the Trolls?

## (FR)
## Quels sont les hashtags les plus couramment utilisés par les Trolls ?

In [ ]:
cypher_query = '''
MATCH (t:Troll)-[:POSTED]->(tw:Tweet)-[:HAS_TAG]->(ht:Hashtag)
RETURN ht.tag, COUNT(tw) AS num
ORDER BY num DESC
'''

result = query_neo4j(driver, cypher_query)

In [ ]:
trollsHashtags = pd.DataFrame(result)

In [ ]:
trollsHashtags.head(20)

## (EN)
## What Troll accounts have the most followers?

## (FR)
## Quels comptes Troll ont le plus de followers ?

In [ ]:
cypher_query = '''
MATCH (u:Troll) 
WHERE EXISTS(u.followers_count)
RETURN u.screen_name AS screen_name, u.followers_count AS followers
ORDER BY followers DESC
'''

result = query_neo4j(driver, cypher_query)

In [ ]:
trollsFollowers = pd.DataFrame(result)

In [ ]:
trollsFollowers.head(10)

## (EN)
## Trolls Dynamics over time 

## (FR)
## Dynamique des trolls au fil du temps

In [ ]:
cypher_query = '''
MATCH (t:Tweet)<-[:POSTED]-(u:Troll) WHERE not(t.created_str='')
WITH apoc.date.parse(t.created_str, "ms", "yyyy-MM-dd HH:mm:ss") as ms
RETURN date(datetime({epochmillis:ms})).year as year, date(datetime({epochmillis:ms})).day as day, date(datetime({epochmillis:ms})).month as month, COUNT(*) as num ORDER BY year, month, day
'''

result = query_neo4j(driver, cypher_query)

In [ ]:
timeline = pd.DataFrame(result)
timeline['date'] = pd.to_datetime(timeline[['year', 'month', 'day']])
timeline = timeline.set_index('date')

In [ ]:
timeline

In [ ]:
fig, ax = plt.subplots()

timeline["num"].plot(ax=ax)

ax.set_ylim(-100, 4000)
ax.yaxis.grid()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)

ax.set_ylabel("number of tweets");

## (EN)
## Inferred relationships — what Trolls are retweeting other Trolls?

## (FR)
## Quels Trolls retweetent d'autres Trolls ? 

In [ ]:
cypher_query = '''
MATCH (r1:Troll)-[:POSTED]->(:Tweet)<-[:RETWEETED]-(:Tweet)<-[:POSTED]-(r2:Troll)
RETURN id(r2) as source, 
    id(r1) as target, 
    r1.screen_name as source_screen_name, 
    r1.name as source_name,
    r2.screen_name as target_screen_name,
    r2.name as target_name,
    COUNT(*) as weight
'''

edges = query_neo4j(driver, cypher_query)

## (EN)
## Build the retweet network with networkx   

## (FR)
## Sauvegarde le réseau de retweet avec networkx

In [ ]:
G = nx.Graph()
for edge in edges:
    if edge["source"] != edge["target"]:
        G.add_node(edge["target"], name=edge["target_screen_name"])
        G.add_node(edge["source"], name=edge["source_screen_name"])
        G.add_edge(edge["source"], edge["target"], weight=edge["weight"])

In [ ]:
G.nodes(data=True)

## (EN)
## Find the communities and compute the pagerank of each troll by using the neo4j API

## (FR)
## Trouvez les communautés et calculez le pagerank de chaque troll en utilisant l'API neo4j

In [ ]:
cypher_query = '''
CALL gds.pageRank.stream({
    nodeQuery:"MATCH (t:Troll) RETURN id(t) AS id", 
    relationshipQuery:"MATCH (r1:Troll)-[:POSTED]->(:Tweet)<-[:RETWEETED]-(:Tweet)<-[:POSTED]-(r2:Troll) RETURN id(r2) as source, id(r1) as target"
})
YIELD nodeId, score
RETURN nodeId, gds.util.asNode(nodeId).screen_name AS name, score ORDER BY score DESC, name ASC
'''

pr = query_neo4j(driver, cypher_query)

In [ ]:
pr

In [ ]:
cypher_query = '''
CALL gds.louvain.stream({
  nodeQuery:"MATCH (t:Troll) RETURN id(t) AS id", 
  relationshipQuery:"MATCH (r1:Troll)-[:POSTED]->(:Tweet)<-[:RETWEETED]-(:Tweet)<-[:POSTED]-(r2:Troll) RETURN id(r2) as source, id(r1) as target, count(*) as weight"
})
YIELD nodeId, communityId AS Community
RETURN nodeId, gds.util.asNode(nodeId).screen_name AS Name, Community
'''

community = query_neo4j(driver, cypher_query)

In [ ]:
community

## (EN)
## Find the communities and compute the pagerank of each troll by using networkx

Use the [Louvain](https://en.wikipedia.org/wiki/Louvain_method) algorithm definied in the [scikit-network](https://scikit-network.readthedocs.io/en/latest/tutorials/clustering/louvain.html) library to calculate the different communities in the retweet network. 

## (FR)
## Trouvez les communautés et calculez le pagerank de chaque troll en utilisant networkx

Utilisez l'algorithme de [Louvain](https://fr.wikipedia.org/wiki/M%C3%A9thode_de_Louvain) et la librairie [scikit-network](https://scikit-network.readthedocs.io/en/latest/tutorials/clustering/louvain.html) pour calculer les différentes communautés contrenue dans le reseau de retweet.  

In [ ]:
from sknetwork.clustering import Louvain
from sknetwork.ranking import PageRank

# 1. extract the largest connected component
largest_cc = max(nx.connected_components(G), key=len)
S = G.subgraph(largest_cc).copy()

# YOUR CODE HERE
raise NotImplementedError()

# Store the result in the graph
results = []
for v in range(len(node_list)):
    results.append([node_list[v], communities[v], pr[node_list[v]], S.nodes[node_list[v]]["name"]])
    S.nodes[node_list[v]]["community"] = communities[v]
    S.nodes[node_list[v]]["pagerank"] = pr[node_list[v]]
    
df = pd.DataFrame(results, columns=["node_id", "community_id", "pagerank", "name"])
df = df.sort_values("pagerank", ascending=False)
df

## (EN)
## Plot the retweet network with the community color coded and the node size proportinal to the pagerank of that node

## (FR)
## Trouvez les communautés et calculez le pagerank de chaque troll en utilisant networkx

In [ ]:
dpi = 300
h,w = 2480,3508

pos = nx.spring_layout(S, weight="weight",k=0.25, iterations=20)

# YOUR CODE HERE
raise NotImplementedError()

fig = plt.figure(figsize=(w/dpi,h/dpi),dpi=dpi)
ax = plt.gca()


nx.draw_networkx_edges(S, pos, alpha=0.3)
nx.draw_networkx_nodes(
        S,
        pos,
        node_color=[S.nodes[node_id]['community'] for node_id in list(S)],
        node_size=node_size,
        edgecolors="white"
)
nx.draw_networkx_labels(S, pos, font_size=7, font_color="white");
plt.axis("off")
plt.show()

## (EN)
## By reading tweets originating from different communities, could you guess what is the main difference between them

## (FR)
## En lisant des tweets provenant de différentes communautés, pourriez-vous deviner quelles sont les principales différence entre les communautés

In [ ]:
S.nodes[44]

In [ ]:
cypher_query = '''
MATCH (r1:Troll)-[:POSTED]->(tw:Tweet)-[:HAS_TAG]->(ht:Hashtag)
WHERE r1.screen_name = 'RealTEN_GOP'
RETURN ht.tag AS tag, COUNT(tw) AS num ORDER BY num DESC
'''
hashtag = query_neo4j(driver, cypher_query)

In [ ]:
hashtag

In [ ]:
cypher_query = '''
MATCH (r1:Troll)-[:POSTED]->(t:Tweet)
WHERE r1.screen_name = 'RealTEN_GOP'
RETURN t.text LIMIT 20
'''

tweets = query_neo4j(driver, cypher_query)

In [ ]:
tweets